In [ ]:
import numpy as np
#from sklearn.decomposition import PCA
import pickle
from keras_unet_collection import models
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow
import pandas as pd
from keras_unet_collection import models

In [ ]:
unet_plus_2d = models.unet_plus_2d((512,512,3), [64,128,256,512], n_labels=2,
                    stack_num_down=2, stack_num_up=2,
                    activation='LeakyReLU', output_activation='Softmax',
                    batch_norm=False, pool='max', unpool=False, deep_supervision=True, name='xnet')

In [ ]:
unet_plus_2d.summary()

In [ ]:
NUM_FOLD = 10

In [ ]:
def my_image_mask_generator(image_data_generator, mask_data_generator):
    train_generator = zip(image_data_generator, mask_data_generator)
    for (img, mask) in train_generator:
        yield (img, mask)

In [ ]:
train_datagen = ImageDataGenerator(rescale =1, shear_range = 0, zoom_range=1,
                                    horizontal_flip = False, validation_split = 0.2)

In [ ]:
BATCH_SIZE,EPOCHS =1, 200

In [ ]:
lr_schedule=ExponentialDecay(0.01,decay_rate=0.8,decay_steps=10000)

In [ ]:
for i in range(1,6):

        #j+=1
        
        x_train_df = pd.read_csv(f'G:/folds_new/train_fold_{i}.csv')
        
        x_valid_df = pd.read_csv(f'G:/folds_new/valid_fold_{i}.csv')
        
        
        
        
        
         
        training_set = train_datagen.flow_from_dataframe(dataframe= x_train_df, directory=f'G:/sen11_cross_validation_image/fold_{i}/train',
                                                 x_col="Image", 
                                                 class_mode=None,
                                                 target_size=(512,512), batch_size=BATCH_SIZE,
                                                 seed = 1)
                                                 
                                                 
        
        training_mask = train_datagen.flow_from_dataframe(dataframe = x_train_df,
                                                  
                                                  directory = f'G:/sen11_cross_validation_mask_editted_2/fold_{i}/train',
                                                  x_col = "Mask", 
                                                  target_size = (512,512),
                                                  class_mode = None,
                                                  batch_size = BATCH_SIZE,
                                                  color_mode = "grayscale",
                                                  seed = 1)
                                                  
        validation_set = train_datagen.flow_from_dataframe(dataframe = x_valid_df, directory = f'G:/sen11_cross_validation_image/fold_{i}/valid',
        
                                                      x_col="Image",
                                                      class_mode=None,
                                                      target_size=(512,512), batch_size=BATCH_SIZE,
                                                      seed=1)  
                                                      
        validation_mask = train_datagen.flow_from_dataframe( dataframe = x_valid_df, directory = f'G:/sen11_cross_validation_mask_editted_2/fold_{i}/valid',
                                                        x_col = "Mask",
                                                        class_mode=None,
                                                        target_size=(512,512), batch_size=BATCH_SIZE,
                                                        color_mode="grayscale",
                                                        seed=1)
         
         
         
                                           
        training_set_generator = my_image_mask_generator(training_set, training_mask)   
        
        validation_set_generator = my_image_mask_generator(validation_set, validation_mask)             
        
              
        unet_plus_2d.compile(loss='binary_crossentropy',optimizer=Adam(learning_rate=lr_schedule),metrics=['accuracy'])
      
        #unet_plus_2d=load_model('D:/Nafiseh/flood_proposal/QC_2019_new_train_data_unet_plus_2d.h5')
      
      
        es=EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)
      
        mch=ModelCheckpoint(filepath=f'G:/Sen1Flood11_data_Unet_plus_2d_fold_{i}.h5',save_best_only=True)
      
        #history_unet_plus_2d=unet_plus_2d.fit(train_resampled__,train_label_unet,batch_size,epochs=epochs,callbacks=[es,mch],validation_data=(val_
      
        history=unet_plus_2d.fit_generator(training_set_generator,
                                          epochs = EPOCHS,
                                          steps_per_epoch=x_train_df.shape[0] // BATCH_SIZE, callbacks=[es, mch], validation_steps = x_valid_df.shape[0], 
                                          validation_data=validation_set_generator)
      
      
        #predicted_segments=np.argmax(unet_plus_2d.predict(test_resampled__)[-1],axis=3)
      
        #gc.collect()
      
        with open(f'G:/training_history_Unet_Plus_2d_Sen1Flood11_data_fold_{i}.pckl','wb') as hist:
      
            pickle.dump(history.history,hist)

In [ ]:
test_datagen = ImageDataGenerator(rescale =1)

In [ ]:
from tensorflow.keras.models import load_model

for i in range(1,6):
    
      unet_plus_2d=load_model(f'G:/Sen1Flood11_data_Unet_plus_2d_fold_{i}.h5')  
    
      x_test_df = pd.read_csv(f'G:/folds_new/test_fold_{i}.csv')
    
    
      test_set = test_datagen.flow_from_dataframe(dataframe=x_test_df, directory=f'G:/sen11_cross_validation_image/fold_{i}/test',
                                                 x_col="Image",
                                                 class_mode=None,
                                                 target_size=(512,512), batch_size=BATCH_SIZE,
                                                 seed =1)
      
      predicted = np.argmax(unet_plus_2d.predict(test_set, x_test_df.shape[0]//BATCH_SIZE)[0],axis=3) 
      np.save(f'G:/predicted_Sen1Flood11_Unet_Plus_2d_fold_{i}.npy',predicted)
      
      

In [ ]:
from osgeo import gdal
import matplotlib.pyplot as plt

img = gdal.Open('G:/nafise_bucket/Pakistan/Pakistan_9684_Pre.tif')

image = img.GetRasterBand(2)

image_ = image.ReadAsArray().astype('float')

plt.imshow(image_)

plt.show()

print(np.min(image_), np.max(image_))